In [ ]:
# Library to fix contractions
!pip install contractions

     |████████████████████████████████| 284 kB 891 kB/s            
     |████████████████████████████████| 106 kB 12.7 MB/s            


In [ ]:
# Module to get variable name as str
!pip3 install varname

In [ ]:
# Load the TensorBoard notebook extension
#%load_ext tensorboard
# Reload TensorBoard
%reload_ext tensorboard

In [ ]:
from datetime import datetime as dt
from varname import argname
import os
import re
import sys

from keras.utils import np_utils
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import zipfile
import contractions

## Loading Data

<br>Text data is obtained from <a href="https://www.gutenberg.org/">Project Gutenberg</a>. This notebook closely follows tutorial by Machine Learning Mastery (Ref [2]).

<br>
<br>
References:
<br>[1] <a href="https://www.gutenberg.org/ebooks/16"> Peter Pan by J. M. Barrie</a>
<br>[2] <a href="https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/">LSTM Text generation tutorial</a>

In [ ]:
# Download Peter Pan from Project Gutenberg Ref[1]
!wget --no-check-certificate \
    "https://www.gutenberg.org/files/16/16-0.zip" \
    -O "/peterpan.zip"

--2022-04-03 14:45:27--  https://www.gutenberg.org/files/16/16-0.zip
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 107629 (105K) [application/zip]
Saving to: ‘/peterpan.zip’

/peterpan.zip       100%[===================>] 105.11K   517KB/s    in 0.2s    

2022-04-03 14:45:28 (517 KB/s) - ‘/peterpan.zip’ saved [107629/107629]



In [ ]:
# Extract dataset to current session
zip_ref = zipfile.ZipFile('/peterpan.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/') #Extracts the files into the /tmp folder
zip_ref.close()

## Preprocessing Data



1.   Clean data.
     - Delete unwanted text from data (eg. table of contents etc).
     - Remove special characters `. , ! ?`.
     - Convert contractions (eg. I've -> I have).
2.   Create mapping of unique characters to integers.
3.   Split processed data into input `X` and output `y` data.
4.   Reshape input data into `[samples, time steps, features]`.
5.   Feature normalization.



#### Clean data

In [ ]:
filename = "/16-0.txt"

In [ ]:
# Get line number at which the story actually starts and ends
for n, line in enumerate(open(filename)):
    if 'Chapter I.\n' in line:
      startl = n + 1
      print('Start at Line', startl)
    if 'THE END\n' in line:
      endl = n + 1
      print('End at Line', endl)


Start at Line 66
End at Line 6288


In [ ]:
# Rewrite the file to only include line from start line to 1 line before end line
with open(filename, 'r+') as f:
    lines = f.readlines()
    # move file pointer to the beginning of a file
    f.seek(0)
    # truncate the file
    f.truncate()

    # start writing lines except the first line
    # lines[1:] from line 2 to last line
    f.writelines(lines[startl:endl])

In [ ]:
# Check if unwanted texts are properly deleted
for n, line in enumerate(open(filename)):
    if 'The Project Gutenberg eBook of Peter Pan, by James M. Barrie' in line:
      print('Oh no! Please check!')
    if 'Section 1. General Terms of Use' in line:
      print('Oh no! Please check!')

In [ ]:
# Load the book into memory
raw_txt = open(filename, 'r', encoding='utf-8').read()
# Convert all to lowercase
raw_txt = raw_txt.lower()

In [ ]:
# Fix contractions
raw_txt = contractions.fix(raw_txt)

# Examining the change
print(raw_txt[2500:3300])

mrs. darling’s bed, holding her hand and calculating expenses,
while she looked at him imploringly. she wanted to risk it, come what
might, but that was not his way; his way was with a pencil and a piece
of paper, and if she confused him with suggestions he had to begin at
the beginning again.

“now do not interrupt,” he would beg of her.

“i have one pound seventeen here, and two and six at the office; i can
cut off my coffee at the office, say ten shillings, making two nine and
six, with your eighteen and three makes three nine seven, with five
naught naught in my cheque-book makes eight nine seven—who is that
moving?—eight nine seven, dot and carry seven—do not speak, my own—and
the pound you lent to that man who came to the door—quiet, child—dot
and carry child—there, you have done it!


In [ ]:
# Delete all special characters and stripped white spaces
raw_txt = ' '.join([re.sub(r"[^a-zA-Z0-9]+", ' ', l).strip() for l in raw_txt.split("\n")])

# Examining the change
print(raw_txt[2500:3300])

 looked at him imploringly she wanted to risk it come what might but that was not his way his way was with a pencil and a piece of paper and if she confused him with suggestions he had to begin at the beginning again  now do not interrupt he would beg of her  i have one pound seventeen here and two and six at the office i can cut off my coffee at the office say ten shillings making two nine and six with your eighteen and three makes three nine seven with five naught naught in my cheque book makes eight nine seven who is that moving eight nine seven dot and carry seven do not speak my own and the pound you lent to that man who came to the door quiet child dot and carry child there you have done it did i say nine nine seven yes i said nine nine seven the question is can we try it for a year 


#### Unique character mapping

In [ ]:
# Create a sorted list of unique characters
chars = sorted(list(set(raw_txt)))
# Create dictionary to store unique character to int pair
char_to_int = {}
for i,v in enumerate(chars):
  char_to_int[v] = i

# Examining the dictionary
print(char_to_int)

{' ': 0, '0': 1, '1': 2, '2': 3, '3': 4, '4': 5, '6': 6, '7': 7, 'a': 8, 'b': 9, 'c': 10, 'd': 11, 'e': 12, 'f': 13, 'g': 14, 'h': 15, 'i': 16, 'j': 17, 'k': 18, 'l': 19, 'm': 20, 'n': 21, 'o': 22, 'p': 23, 'q': 24, 'r': 25, 's': 26, 't': 27, 'u': 28, 'v': 29, 'w': 30, 'x': 31, 'y': 32, 'z': 33}


In [ ]:
total_chars = len(raw_txt)
total_unique_chars = len(chars)
print("Total characters in the whole book: ", total_chars)
print("Total unique characters: ", total_unique_chars)

Total characters in the whole book:  244253
Total unique characters:  34


#### Input/ Output split


- `seq_length` determine how the data will be split. Essentially, the length of every subsequence.
- Each training set will comprise of *n* time steps of 1 character input `X` followed by 1 character output `y`.
- For example, if the sequence length is 4, then the training pattern would be:
    <br>`Input -> Output`
    <br>` SUND -> A`
    <br>` UNDA -> Y`






In [ ]:
seq_length = 200
data_X = []
data_y = []

for i in range(0, total_chars - seq_length, 1):
    seq_in = raw_txt[i : i+seq_length] # slice for every 200 characters for each split starting from i
    seq_out = raw_txt[i+seq_length] # the next character after seq_in
    
    for char in seq_in:
        data_X.append(char_to_int[char]) # get the int value of each character and append to X
        
    data_y.append(char_to_int[seq_out]) # append the only character in seq_out

In [ ]:
total_patterns = len(data_X)
print('Total patterns:', total_patterns)

Total patterns: 48810600


#### Reshape into `[samples, time steps, features]`

- samples = number of patterns we have (here we use `-1` to let numpy infer the number of rows)
- time steps = number of sequence length per split
- features = 1 dimension feature (to produce a 2D array)

In [ ]:
X = np.reshape(data_X, (-1, seq_length, 1)).astype(np.float32)
print(X.shape)

(244053, 200, 1)


#### Normalization

- Rescale the integers to the range 0 to 1.
- One-hot encode the output variable to configure the network to predict the probability of the output being each of the 34 unique characters previously identified.
- That is, each `y` vlaue is converted into a sparse vactor with a length of 23, full of zeros except with a 1 in the column for the predicted letter.

In [ ]:
# Normalize
X = X / float(total_unique_chars)

# One hot encode the output variable
y = np_utils.to_categorical(data_y).astype(np.float32)

## Build Model

Simple LSTM architecture: **3** stacked LSTM Layers with 0.1 Dropout.

In [ ]:
# Build CNN Model

model = tf.keras.Sequential([
                                tf.keras.layers.LSTM(255, input_shape=(X.shape[1], X.shape[2]), return_sequences=True),
                                tf.keras.layers.LSTM(255, return_sequences=True),
                                tf.keras.layers.LSTM(255, dropout=0.1),
                                tf.keras.layers.Dense(y.shape[1], activation='softmax')
])

model.summary()

2022-04-03 14:45:44.246153: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-03 14:45:44.338867: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-03 14:45:44.339621: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-03 14:45:44.340807: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 200, 255)          262140    
_________________________________________________________________
lstm_1 (LSTM)                (None, 200, 255)          521220    
_________________________________________________________________
lstm_2 (LSTM)                (None, 255)               521220    
_________________________________________________________________
dense (Dense)                (None, 34)                8704      
Total params: 1,313,284
Trainable params: 1,313,284
Non-trainable params: 0
_________________________________________________________________


## Train Model

In [ ]:
def train_model(model, epochs=100):
    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

    # Initialize tensorboard
    logdir = os.path.join("logs", argname('model'))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
    
    # Initialize earlystopping
    earlystop_callback = tf.keras.callbacks.EarlyStopping(patience=1, verbose=0, monitor='accuracy', restore_best_weights=True)
    
    # Initialize checkpoint
    filepath = os.path.join("checkpoint", "weights-improvement-{epoch:02d}-{accuracy:.2f}.hdf5")
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')

    return model.fit(X, y, epochs=epochs, batch_size=128, callbacks=[tensorboard_callback, checkpoint_callback, earlystop_callback])



def plot_hist(hist):
    plt.plot(hist.history["auccuracy"])
    plt.title(f"{argname('hist')[:-8]} performance")
    plt.ylabel("area_under_curve")
    plt.xlabel("epoch")
    plt.legend(["train accuracy"], loc="upper left")
    plt.show()

In [ ]:
history = train_model(model)

## Observe model on TensorBoard

In [ ]:
%tensorboard --logdir logs

## Conclusion


1.   Trained for a total of 49 epochs until accuracy stops improving.
2.   Best accuracy obtained is 71%.

Note: Due to *choppy* training time, model training was done in stages.

## Deploy Model!

View the text generated by the trained model.

In [ ]:
int_to_char = dict((i, c) for i, c in enumerate(chars))
start_seq = np.random.randint(0, len(data_X)-1)
pattern = data_X[start_seq : start_seq+seq_length]
print('Seed:')
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate 1000 characters
for i in range (1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x/ float(total_unique_chars)
    pred = model.predict(x, verbose=0)
    rand = pred[np.random.choice(pred.shape[0], 1, replace=False), :]
    index = np.argmax(rand)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print('\nEnd of prediction')

Seed:
" ide and sometimes winning and when it won peter always sympathetic to the weaker side could not help clapping it was such a gallant piece of paper  it was not really a piece for it was fighting the ti "
me the children are all the boys were all but i will tell you  we could go the most beautiful father went again in the plank to wendy he said the pirates started on the water that the boys were grown up the last time they were all on the rock wendy s fault  she was frightfully ashameep the gaiety of the bed post and then at last he stood on the rock and she saw that the boys were grown up the last time they were all on the rock wendy s fault  she was frightfully ashameep the gaiety of the bed post and then at last he stood on the rock and she saw that the boys were grown up the last time they were all on the rock wendy s fault  she was frightfully ashameep the gaiety of the bed post and then at last he stood on the rock and she saw that the boys were grown up the last time they 

## Load Model

In [ ]:
model = tf.keras.models.load_model('../input/peterpan-model-71/weights-improvement-14-0.71.hdf5')

# Check its architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 200, 255)          262140    
_________________________________________________________________
lstm_1 (LSTM)                (None, 200, 255)          521220    
_________________________________________________________________
lstm_2 (LSTM)                (None, 255)               521220    
_________________________________________________________________
dense (Dense)                (None, 34)                8704      
Total params: 1,313,284
Trainable params: 1,313,284
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances
# notebook.display(port=6006, height=5000)